In [27]:
from pathlib import Path
import pycolmap

# Setup paths
images = Path('images')
outputs = Path('outputs')
outputs.mkdir()
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'

# 1. SIFT + COLMAP pipeline
def run_sift_colmap():
    database_path = outputs / "database.db"

    # Extract SIFT features and match
    pycolmap.extract_features(database_path, images)
    pycolmap.match_exhaustive(database_path)

    # Run reconstruction
    maps = pycolmap.incremental_mapping(database_path, images, outputs)
    return maps[0]


FileExistsError: [Errno 17] File exists: 'outputs'

In [1]:
import numpy as np
import pycolmap
from PIL import Image, ImageOps
import torch
import time
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import cv2
from superpoint_superglue_deployment import Matcher

In [ ]:
def extract_sift_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img = ImageOps.grayscale(img)
    img = np.array(img).astype(np.float) / 255.

    sift = pycolmap.Sift()
    start_time = time.time()
    keypoints, descriptors = sift.extract(img)
    end_time = time.time()

    return keypoints, descriptors, end_time - start_time

def extract_superpoint_features(image_path):
    start_time = time.time()
    query_image = cv2.imread("images/1.png", 0)
    ref_image = cv2.imread("images/2.png", 0)



    superglue_matcher = Matcher(
        {
            "superpoint": {
                "input_shape": (-1, -1),
                "keypoint_threshold": 0.003,
            },
            "superglue": {
                "match_threshold": 0.5,
            },
            "use_gpu": False,
        }
    )
    query_kpts, ref_kpts, _, _, matches = superglue_matcher.match(query_image, ref_image)
    end_time = time.time()

    return query_kpts, ref_kpts, matches, end_time - start_time

def compute_metrics_superglue(query_kpts, ref_kpts, matches):
    valid_matches = [m for m in matches if m.queryIdx != -1]

    # Extract matched keypoints using DMatch indices
    matched_query_kpts = np.array([query_kpts[m.queryIdx] for m in valid_matches])
    matched_ref_kpts = np.array([ref_kpts[m.trainIdx] for m in valid_matches])

    metrics = {
        'SuperPoint+SuperGlue': {
            'num_keypoints': len(query_kpts),
            'num_matches': len(valid_matches),
            'matching_ratio': len(valid_matches) / len(query_kpts),
            # 'spatial_distribution': compute_spatial_distribution(matched_query_kpts),
            # 'mean_distance': compute_mean_distance(matched_query_kpts, matched_ref_kpts)
        }
    }
    return metrics

In [26]:
def evaluate_superpoint(image_path1, image_path2):
  # Extract features from both images
  query_kp1, ref_kp1, matches, time1 = extract_superpoint_features(image_path1)
  print(matches[0])
  # kp2, desc2, time2 = extract_superpoint_features(image_path2)
  metrics = compute_metrics_superglue(query_kp1, ref_kp1, matches)
  print(metrics)

def compute_metrics_sift(query_image, ref_image):
    query_image = cv2.imread("images/1.png", 0)
    ref_image = cv2.imread("images/2.png", 0)

    sift = pycolmap.Sift()
    # Extract SIFT features

    start_time = time.time()
    q_kpts, q_desc = sift.extract(query_image)
    r_kpts, r_desc = sift.extract(ref_image)

    # Match SIFT features using nearest neighbor
    tree = cKDTree(r_desc)
    distances, indices = tree.query(q_desc, k=2)

    # Lowe's ratio test
    ratio_threshold = 0.8
    valid_matches = distances[:, 0] < ratio_threshold * distances[:, 1]
    matched_query_kpts = q_kpts[valid_matches]
    matched_ref_kpts = r_kpts[indices[valid_matches, 0]]

    metrics = {
        'SIFT': {
            'num_keypoints': len(q_kpts),
            'num_matches': np.sum(valid_matches),
            'matching_ratio': np.sum(valid_matches) / len(q_kpts),
            # 'mean_distance': compute_mean_distance(matched_query_kpts, matched_ref_kpts)
        }
    }
    print(metrics)
    return metrics


compute_metrics_sift('images/1.png', 'images/2.png')
evaluate_superpoint('images/1.jpg', 'images/2.jpg')

/var/folders/qr/c_8vj3d52fl13hwpqb7419nc0000gn/T/ipykernel_33532/3747475331.py:13: DeprecationWarning: No SIFT extraction options specified. Setting them to peak_threshold=0.01, first_octave=0, max_image_size=7000 for backwards compatibility. If you want to keep the settings, explicitly specify them, because the defaults will change in the next major release.
  sift = pycolmap.Sift()
I20241212 15:12:01.953156 0x1f7d07ac0 sift.cc:727] Creating SIFT CPU feature extractor
/Users/carterweaver/envs/18794/lib/python3.11/site-packages/superpoint_superglue_deployment/superpoint_handler.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will

{'SIFT': {'num_keypoints': 1164, 'num_matches': np.int64(656), 'matching_ratio': np.float64(0.563573883161512)}}
< cv2.DMatch 0x148c42870>
{'SuperPoint+SuperGlue': {'num_keypoints': 952, 'num_matches': 647, 'matching_ratio': 0.6796218487394958}}
